In [4]:
library(monocle)
library(mvtnorm)
library(igraph)
library(reshape2)
#导入函数
setwd("C:/Users/jliu25/Desktop/Unknown/functions")
source('functions.r')

In [ ]:
#构建cellData类型数据
library(HSMMSingleCell)
data(HSMM_expr_matrix) ## RPKM 矩阵,271个细胞，47192个基因
data(HSMM_gene_annotation)
data(HSMM_sample_sheet)
pd <- new("AnnotatedDataFrame", data = HSMM_sample_sheet)
fd <- new("AnnotatedDataFrame", data = HSMM_gene_annotation)
HSMM <- newCellDataSet(as.matrix(HSMM_expr_matrix),   
                       phenoData = pd, 
                       featureData = fd,
                       lowerDetectionLimit=0.1,
                       expressionFamily=tobit(Lower=0.1))
rpc_matrix <- relative2abs(HSMM)
HSMM <- newCellDataSet(as(as.matrix(rpc_matrix), "sparseMatrix"),
                       phenoData = pd, 
                       featureData = fd,
                       lowerDetectionLimit=0.5,
                       expressionFamily=negbinomial.size())

In [ ]:
#monocle2复现
#数据预处理
HSMM <- estimateSizeFactors(HSMM)
HSMM <- estimateDispersions(HSMM)
HSMM <- detectGenes(HSMM, min_expr = 0.1)
## 对每个基因都检查一下在多少个细胞里面是有表达量的。
## 只留下至少在10个细胞里面有表达量的那些基因，做后续分析
expressed_genes <- row.names(subset(fData(HSMM), num_cells_expressed >= 10))

pData(HSMM)$Total_mRNAs <- Matrix::colSums(exprs(HSMM))
HSMM <- HSMM[,pData(HSMM)$Total_mRNAs < 1e6]
upper_bound <- 10^(mean(log10(pData(HSMM)$Total_mRNAs)) +
                     2*sd(log10(pData(HSMM)$Total_mRNAs)))
lower_bound <- 10^(mean(log10(pData(HSMM)$Total_mRNAs)) -
                     2*sd(log10(pData(HSMM)$Total_mRNAs)))
table(pData(HSMM)$Hours)
qplot(Total_mRNAs, data = pData(HSMM), color = Hours, geom = "density") +
  geom_vline(xintercept = lower_bound) +
  geom_vline(xintercept = upper_bound)
HSMM <- HSMM[,pData(HSMM)$Total_mRNAs > lower_bound & 
               pData(HSMM)$Total_mRNAs < upper_bound]                                 
HSMM <- detectGenes(HSMM, min_expr = 0.1)
L <- log(exprs(HSMM[expressed_genes,]))
melted_dens_df <- melt(Matrix::t(scale(Matrix::t(L))))
qplot(value, geom="density", data=melted_dens_df) +  stat_function(fun = dnorm, size=0.5, color='red') + 
  xlab("Standardized log(FPKM)") +
  ylab("Density")

#细胞分类，挑出Myoblast
## 根据基因名字找到其在表达矩阵的ID，这里是ENSEMBL数据库的ID
MYF5_id <- row.names(subset(fData(HSMM), gene_short_name == "MYF5"))
ANPEP_id <- row.names(subset(fData(HSMM), gene_short_name == "ANPEP"))
## 这里选取的基因取决于自己的单细胞实验设计
cth <- newCellTypeHierarchy()

cth <- addCellType(cth, "Myoblast", classify_func = function(x) { x[MYF5_id,] >= 1 })
cth <- addCellType(cth, "Fibroblast", classify_func = function(x)
{ x[MYF5_id,] < 1 & x[ANPEP_id,] > 1 })

HSMM <- classifyCells(HSMM, cth, 0.1)
## 这个时候的HSMM已经被改变了，增加了属性。

table(pData(HSMM)$CellType)

pie <- ggplot(pData(HSMM), aes(x = factor(1), fill = factor(CellType))) +
  geom_bar(width = 1)
pie + coord_polar(theta = "y") +
  theme(axis.title.x = element_blank(), axis.title.y = element_blank())

marker_diff <- markerDiffTable(HSMM[expressed_genes,],
                               cth,
                               residualModelFormulaStr = "~Media + num_genes_expressed",
                               cores = 1)
candidate_clustering_genes <- row.names(subset(marker_diff, qval < 0.01))
marker_spec <- calculateMarkerSpecificity(HSMM[candidate_clustering_genes,], cth)
head(selectTopMarkers(marker_spec, 3))
semisup_clustering_genes <- unique(selectTopMarkers(marker_spec, 500)$gene_id)
HSMM <- setOrderingFilter(HSMM, semisup_clustering_genes)
plot_ordering_genes(HSMM)
plot_pc_variance_explained(HSMM, return_all = F)
HSMM <- reduceDimension(HSMM, max_components = 2, num_dim = 3,
                        norm_method = 'log',
                        reduction_method = 'tSNE',
                        residualModelFormulaStr = "~Media + num_genes_expressed",
                        verbose = T)
HSMM <- clusterCells(HSMM, num_clusters = 2)
plot_cell_clusters(HSMM, 1, 2, color = "CellType")

HSMM_myo <- HSMM
diff_test_res <- differentialGeneTest(HSMM_myo[expressed_genes,],
                                      fullModelFormulaStr = "~Media")
ordering_genes <- row.names (subset(diff_test_res, qval < 0.01))
HSMM_myo <- setOrderingFilter(HSMM_myo, ordering_genes)
plot_ordering_genes(HSMM_myo)

HSMM_myo <- reduceDimension(HSMM_myo, max_components = 2,
                            method = 'DDRTree')
HSMM_myo <- orderCells(HSMM_myo)
plot_cell_trajectory(HSMM_myo, color_by = "Hours")
plot_cell_trajectory(HSMM_myo, color_by = "State")

In [ ]:
HSMM_myo <- reduceDimension(HSMM, max_components=2)
k=4
pi = c(1/4,1/4,1/4,1/4)
data<-t(reducedDimS(HSMM_myo))
initialpara<-initialGMMs(data,k)
mu<-initialpara[["mu"]]
sigma<-initialpara[["sig"]]

cds<-HSMM_myo
count=1
while (TRUE){
    cds <- project2MST(cds, project_point_to_line_segment)
    newdata<-cds@auxOrderingData[["DDRTree"]]$pr_graph_cell_proj_dist
    updatepara<-mulGaussCluster(t(newdata),k,pi,mu,sigma)
    mu<-updatepara[["mu"]]
    sigma<-updatepara[["sigma"]]
    pi<-updatepara[["pi"]]
    count<-count+1
    if(count == 100){
      break
    }
}

r<-updatepara[["r"]]
cluster <- which(r == apply(r,1,max),arr.ind = TRUE)
cluster <- cluster[order(cluster[,1]),]
#pData(cds)$State<-cluster[,2]
pData(cds)$cluster<-as.character(cluster[,2])
plot_cell_trajectory(cds, color_by="cluster")

#true state
plot_cell_trajectory(cds, color_by="State")
plot_cell_trajectory(HSMM_myo, color_by = "Hours")

Data 2 test

In [ ]:
#x<-read.table("C:/Users/jliu25/Desktop/Unknown/data/GSE60781_RPKM.txt", header = TRUE, sep = "\t", row.names=1)
load('C:/Users/jliu25/Desktop/Unknown/data/gene_annotation.Rdata')
load('C:/Users/jliu25/Desktop/Unknown/data/cell_annotation.Rdata')
load('C:/Users/jliu25/Desktop/Unknown/data/expression.Rdata')

#构建cellData类型数据
pd <- new("AnnotatedDataFrame", data = data.frame(c))
fd <- new("AnnotatedDataFrame", data = data.frame(g))
cds <- newCellDataSet(as.matrix(x), phenoData = pd, featureData = fd,expressionFamily = tobit())

cds <- estimateSizeFactors(cds)
cds<- detectGenes(cds, min_expr = 0.1)
expressed_genes <- row.names(subset(fData(cds), num_cells_expressed >= 10))
diff_test_res <- differentialGeneTest(cds[expressed_genes,],
+                    fullModelFormulaStr = "~celltype")
ordering_genes <- row.names (subset(diff_test_res, qval < 0.01))
cds <- setOrderingFilter(cds, ordering_genes)


# ----------以下是functions--------------

In [ ]:
mulGaussCluster <- function(data,k,pi,mu,sigma){
  #data:数据集,向量或数据框
  #k:聚类的个数，或高斯分布的个数
  #pi:高斯分布的先验概率,选择各个高斯分布的概率,向量
  #mu:高斯分布的初始均值
  #sigma:多元高斯分布的初始协方差阵
    N = nrow(data)
    u0 <- mu
    p = c()
    for(j in 1:k){
      prob = apply(data,1,dmvnorm,mean=mu[j,],sigma = sigma[[j]])
      p = cbind(p,prob)
    }
    amatrix <- matrix(rep(pi,N),nrow=N,byrow = TRUE)
    r = (p * amatrix) / rowSums(p * amatrix)  
    #updata mu
    mu <- t(r) %*% data / colSums(r) #求和可以转化为向量相乘的形式，简化计算
    #updata sigma
    for(j in 1:k){
      upsigma = matrix(0,ncol(data),ncol(data))
      for(i in 1:N){
        upsigma = upsigma + r[i,j] * (data[i,]-u0[j,]) %*% t(data[i,]-u0[j,])  #R中向量默认为列向量
      }
      sigma[[j]]= upsigma/sum(r[,j]) 
    }
    #updata pi
    pi <- colSums(r)/N
    return(list(mu = mu,sigma = sigma,pi = pi,r=r))
}

In [ ]:
#use kmeans to get initial parameters of GMMs
initialGMMs<-function(data,k){
  km<-kmeans(data,k)
  cluster<-km[["cluster"]]
  mu<-matrix(0,k,ncol(data))
  sigma<-list()
  for (i in 1:k){
    subdata<-data[which(cluster==i),]
    m<-apply(subdata,2,mean)
    s<-cov(subdata)
    mu[i,]<-m
    sigma[[i]]<-s
  }
  initialpara<-list(mu=mu,sig=sigma)
  return (initialpara)
}

# ----------以下未用到----------------

In [16]:
####################
library(mclust) #diabetes data
data(diabetes)
data <- diabetes[,-1]
k<-3
initialpara<-initialGMMs(data,3)
pi = c(1/3,1/3,1/3)
mu<-initialpara[["mu"]]
sigma<-initialpara[["sig"]]

paras<-mulGaussCluster(data,k,pi,mu,sigma)
centers<-ncenter(data,cluster)
ew<-Edge_weight(centers,k)
g <- make_graph(t(ew[,1:2]),directed = FALSE)
m<-mst(g)
#####################

In [ ]:
#计算聚类中心
ncenter<-function(data,cluster){
    centers<-matrix(0,k,ncol(data))
    for (i in 1:k){
    m<-which(cluster[,2]==i)
    centers[i，]<-apply(data[m,],2,mean)
    }
    return(centers)
}

In [ ]:
#计算边权矩阵
Edge_weight<-function(centers,k){
    row<-nrow(centers)*(nrow(centers)-1)/2
    ew<-matrix(0,row,3)
    count<-0
    dis<-as.matrix(dist(centers))
    for (i in 1:(k-1)){
        for (j in (i+1):k){
            count<-count+1
            ew[count,1]=i
            ew[count,2]=j
            ew[count,3]=dis[i,j]
        }
    }
    return(ew)
}